# Data Ingestion

In [2]:
from langchain_core.documents import Document

In [4]:
documents = [
    Document(page_content="Apples are red", metadata={"title": "apple_book","pages":1,"author":"Adil Khan"}),
    Document(page_content="Blueberries are blue", metadata={"title": "blueberry_book","pages":1,"author":"Adil Khan"}),
    Document(page_content="Bananas are yelow", metadata={"title": "banana_book","pages":1,"author":"Adil Khan"}),
]

In [5]:
documents

[Document(metadata={'title': 'apple_book', 'pages': 1, 'author': 'Adil Khan'}, page_content='Apples are red'),
 Document(metadata={'title': 'blueberry_book', 'pages': 1, 'author': 'Adil Khan'}, page_content='Blueberries are blue'),
 Document(metadata={'title': 'banana_book', 'pages': 1, 'author': 'Adil Khan'}, page_content='Bananas are yelow')]

In [6]:
import os
os.makedirs("../data/text_files",exist_ok=True)

In [7]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [10]:
from langchain.document_loaders import TextLoader

loader=TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")

doc=loader.load()

doc

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]

# Embedding and Vector Store

In [1]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
import torch
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

/home/adil-khan/Builds/RAG/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Check GPU's CUDA Core

In [14]:
import torch
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version (PyTorch built against):", torch.version.cuda)
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


PyTorch: 2.8.0+cu128
CUDA available: True
CUDA version (PyTorch built against): 12.8
GPU name: NVIDIA GeForce GTX 1650


### Class to Load Model & create Embeddings

In [ ]:
class EmbeddingManager:
    """Handles document embedding generation using Sentence Transformer (384 dimensions)"""

    def __init__(self,model_name:str="all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args: model_name: Huggingface model name for sentence embeddings
        """
        self.model_name=model_name
        self.model=None
        self._load_model()

    def _load_model(self):
        """Load Sentence Transformer Model"""
        try:
            print(f"Loading Sentence Transformer Model:{self.model_name}")
            device = "cuda" if torch.cuda.is_available() else "cpu"
            print(f"Loading Model into {device}")
            self.model = SentenceTransformer(self.model_name, device=device)
            print(f"Model Loaded sucessfully,Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error in loading model {self.model_name}: {e}")
            raise
    
    def generate_embeddings(self,texts: List[str])->np.ndarray:
        """
        Generate embeddings for a list of text

        Args:
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts),embedding_dim)
        """

        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")

        embeddings=self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    
    def get_embedding_dimension(self)-> int:
        """Get the embedding dimension of model"""

        if not self.model:
            raise ValueError("Model not loaded")
        
        return self.model.get_sentence_embedding_dimension()
    
    ##Initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading Sentence Transformer Model:all-MiniLM-L6-v2
Model Loaded sucessfully,Embedding dimention: 384


### Vector Store